In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from urllib.parse import quote as urlquote
import pandas as pd
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
import numpy as np
import plotly.figure_factory as ff
import plotly.express as px
import pyautogui
import re
from sklearn.metrics import confusion_matrix
from datetime import datetime
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
import operator
from flask import Flask, send_from_directory
import glob
import win32com.client
import numpy as np 
from wordcloud import WordCloud, STOPWORDS
import os
import cv2 
import pyautogui
from PIL import Image
from plotly.subplots import make_subplots
import random
import pythoncom

#pythoncom.CoInitialize()

server = Flask(__name__)

#app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])
app = dash.Dash(__name__,meta_tags=[{"name": "viewport", "content": "width=device-width"}])

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']



#df = pd.read_csv('a.csv',encoding = 'latin-1')
df = pd.read_csv('C:/Users/samarth152721/Desktop/POC/Dashboard/Top 30 banks till 17 July 2020.csv',encoding = 'latin-1')
df['Date'] = pd.to_datetime(df['Date received'])
df['Year'] = pd.to_datetime(df['Date received']).dt.year
df = df[df['Month']!='Dec 2018']

available_indicators = np.array(list(df['State'].unique())+['All'])
available_indicators1 = np.array(list(df['Month'].unique())+['All'])
corona_indicator = np.array(list(df['Corona Flag'].unique())+['All'])
months_indicator = np.array(list(df['Month'].unique()))
product_indicator = np.array(list(df['Product'].unique())+['All'])
multi_months_indicator = np.array(list(df['Month'].unique())+['All'])

button_clicks = 0

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='1. YTD Comparision', children=[
           html.Div([
                html.Div([
#                     html.Div([
#                         html.Button('Reset Graphs', id='button',n_clicks=0),
#                         html.H3(id='button-clicks')
#                     ],
#                         style={'width': '19%','color':"#FFFFFF",'align':'right', 'textAlign': 'right','float':'right'}),
                    html.Div(
    [html.H3(id='dummy'),html.H3(id='table1'),
        dbc.Button("Send Email", id="open"),
        dbc.Modal(
            [
                dbc.ModalHeader("Please enter comma (,) seperated emails"),
                dbc.ModalBody(dbc.Input(id="email", placeholder="Email Ids", type="text")),
                dbc.ModalFooter(
                    dbc.Button("Send", id="send", className="ml-auto")
                ),
            ],
            id="modal",
        ),
    ],
style={'color':"#FFFFFF",'align':'right', 'textAlign': 'right','float':'right'}),
#                         html.Div([
#                             html.H1('Complaints Dashboard')
#                         ],
#                         style={'width': '79%','color':"#000000",'align':'center', 'textAlign': 'center','float':'center'}),
        html.Div([
            dbc.Card([
                html.H5("State", className="card-title"),
                dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='All',
                multi = True
            ),          
       
     ],color = 'light')], style={'width': '46%', 'float': 'center', 'display': 'inline-block' ,'padding': '10px 0px 0px 0px'}),
                    
#         html.Div([
#             dbc.Card([
#             html.P("State: ")
#         ],style={'width':'5%','display': 'inline-block','color':'#000000' ,'padding': '0px 0px 0px 10px'}),
#         html.Div([
#             #html.P('State Selection',style={'display': 'inline-block','color':'#000000'}),
#             dcc.Dropdown(
#                 id='crossfilter-xaxis-column',
#                 options=[{'label': i, 'value': i} for i in available_indicators],
#                 value='All',
#                 multi = True,
#                 #placeholder = 'States'
#             ),
           
#         ],
#         style={'width': '25%', 'display': 'inline-block','color':'#000000' ,'padding': '10px 0px 0px 0px'}),

        html.Div([
            dbc.Card([
             html.H5("Month", className="card-title"),
                dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='All',
                multi = False
            ),        
     ],color = 'light')], style={'width': '46%', 'float': 'center', 'display': 'inline-block' ,'padding': '5px 0px 0px 5px'}),
#             html.P("Month: ")
#         ],style={'width':'5%','display': 'inline-block','color':'#000000' ,'padding': '0px 0px 0px 10px'}),
#         html.Div([
#             dcc.Dropdown(
#                 id='crossfilter-yaxis-column',
#                 options=[{'label': i, 'value': i} for i in available_indicators1],
#                 value='All',
#                 multi = False
#             ),           
#         ], style={'width': '25%', 'float': 'center', 'display': 'inline-block' ,'padding': '5px 0px 0px 10px'}),
            
#         html.Div(dcc.RangeSlider(
#         id='crossfilter-year--slider',
#         min=df['Year'].min(),
#         max=df['Year'].max(),
            
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': '#FFFFFF',
        'padding': '0px 0px'
    }),
        
    dbc.Row(  
        [             
        dbc.Col(html.Div([
        dbc.Card([
         html.H5("Number of Complaints", className="card-title"),
        html.H5(id='total_complaints', className="card-title")
         #dcc.Graph(id = 'stackedbarchart'),
       
     ],color = 'info')],
    style={'height':'12%','padding': '7px 20px 30px 60px','align':'center' ,'float': 'center'}),width = 4),
            
            
            
        dbc.Col(html.Div([
        dbc.Card([
         html.H5("Corona Complaints", className="card-title"),
        html.H5(id='number_of_corona_complaints', className="card-title")
         #dcc.Graph(id = 'stackedbarchart'),
       
     ],color = 'danger')],
    style={'height':'12%','padding': '7px 20px 30px 60px','align':'center' ,'float': 'center'}),width = 4),
    
            dbc.Col(html.Div([
        dbc.Card([
         html.H5("Non Corona Complaints", className="card-title"),
        html.H5(id='number_of_non_corona_complaints', className="card-title")
         #dcc.Graph(id = 'stackedbarchart'),
       
     ],color = 'warning')],
    style={'height':'12%','padding': '7px 20px 30px 60px'}),width = 4),
        
        ],style={'padding': '7px 0px 0px 10px','align':'center' ,'float': 'center'}),
                
            
    dbc.Row(
        [
            
           
    dbc.Col(html.Div([
        dbc.Card([
         #html.H5("Number of Complaints", className="card-title"),
         dcc.Graph(id = 'stackedbarchart'),
       
     ])],
    style={'height':'12%','padding': '0px 0px 0px 30px'}),width = "auto"),
            
            
            dbc.Col(html.Div([
                    dbc.Card([
                        html.H5("Word Cloud for Non Corona Complaints", className="card-title"),
                    #html.Img(src= app.get_asset_url('Word.png'))
                        html.Img(id = 'wordcloud_non_corona')
                ])], 
    style={'backgroundColor': '#FFFFFF','height':'45%','width':'100%','padding': '10px 10px 10px 10px'}),width = "auto"),
            
             dbc.Col(html.Div([
                    dbc.Card([
                        html.H5("Word Cloud for Corona Complaints", className="card-title"),
                    #html.Img(src= app.get_asset_url('Word.png'))
                        html.Img(id = 'wordcloud_corona')
                ])], 
    style={'backgroundColor': '#FFFFFF','height':'45%','width':'100%','padding': '10px 10px 10px 10px'}),width = "auto"),
            
                          
            ])])]),
            
                
    dcc.Tab(label='2. Two Months Together', children=[
        html.Div([
        html.Div([
            dbc.Card([
                html.H5("First Month", className="card-title"),
                dcc.Dropdown(
                id='first_months_column',
                options=[{'label': i, 'value': i} for i in months_indicator],
                value='Mar 2020',
                multi = False
            ),          
       
     ],color = 'light')], style={'width': '49%', 'float': 'center', 'display': 'inline-block' ,'padding': '10px 0px 0px 7px'}),
                    
#             html.Div([
#             dcc.Dropdown(
#                 id='first_months_column',
#                 options=[{'label': i, 'value': i} for i in months_indicator],
#                 value='Feb 2020',
#                 multi = False
#             )], style={'width': '49%', 'float': 'center', 'display': 'inline-block' ,'padding': '10px 0px 0px 10px'}
#             ),
            
            html.Div([
                dbc.Card([
                html.H5("Prior Month", className="card-title"),
                dcc.Dropdown(
                id='second_months_column',
                options=[{'label': i, 'value': i} for i in months_indicator],
                value='Feb 2020',
                multi = False
            ),          
       
     ],color = 'light')], style={'width': '49%', 'float': 'center', 'display': 'inline-block' ,'padding': '10px 0px 0px 7px'}),
       
                
                
                
#             dcc.Dropdown(
#                 id='second_months_column',
#                 options=[{'label': i, 'value': i} for i in months_indicator],
#                 value='Mar 2020',
#                 multi = False
#             )],style={'width': '49%', 'float': 'center', 'display': 'inline-block' ,'padding': '10px 0px 0px 10px'}
#             ),
            
            
      html.Div([      
    dbc.Row([
                
                dbc.Col(html.Div([
        dbc.Card([
            html.H5("Waterfall for both Months", className="card-title"),
         dcc.Graph(id = 'bar_line'),
       
     ])],
    style={'backgroundColor': '#FFFFFF','height':'35%','width':'100%','padding': '10px 20px 30px 20px'}),width = "auto"),
                
                dbc.Col(html.Div([
        dbc.Card([
            html.H5("Top 3 Issues for all Products", className="card-title"),
        dcc.Graph(id = 'tab2_table',config={
         'displayModeBar': False
     }),
       
     ])],
    style={'height':'12%','width':'100%','padding': '10px 20px 30px 20px'}),width = "auto"),
                
                
            ]),])
            
#      html.Div([
#         dcc.Graph(id = 'bar_line'),
#     ],style={'display': 'inline-block', 'width': '100%','height':'20%'}),
            
#             html.Div([
#         dcc.Graph(id = 'tab2_table'),
#     ],style={'display': 'inline-block', 'width': '100%','height':'20%'}),
    
   
    
])]),
    

    
    dcc.Tab(label='3. Month on Month Deep Dive', children=[
        html.Div([
            
            dbc.Card([
                html.H5("State", className="card-title"),
                dcc.Dropdown(
                id='state_column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='All',
                multi = True
            ),          
       
     ],color = 'light')], style={'width': '33%', 'float': 'center', 'display': 'inline-block' ,'padding': '10px 0px 0px 7px'}),
#             dcc.Dropdown(
#                 id='state_column',
#                 options=[{'label': i, 'value': i} for i in available_indicators],
#                 value='All',
#                 multi = True
#             ),           
#         ], style={'width': '33%', 'float': 'center', 'display': 'inline-block' ,'padding': '0px 0px 0px 10px'}),
        html.Div([
            dbc.Card([
                html.H5("Product", className="card-title"),
                dcc.Dropdown(
                id='product_column',
                options=[{'label': i, 'value': i} for i in product_indicator],
                value='All',
                multi = True
            ),          
       
     ],color = 'light')], style={'width': '33%', 'float': 'center', 'display': 'inline-block' ,'padding': '10px 0px 0px 7px'}),
#             dcc.Dropdown(
#                 id='product_column',
#                 options=[{'label': i, 'value': i} for i in product_indicator],
#                 value='All',
#                 multi = True
#             ),           
#         ], style={'width': '33%', 'float': 'center', 'display': 'inline-block' ,'padding': '0px 0px 0px 10px'}),
        
        html.Div([
            dbc.Card([
                html.H5("Month", className="card-title"),
                dcc.Dropdown(
                id='month_column',
                options=[{'label': i, 'value': i} for i in multi_months_indicator],
                value='All',
                multi = True
            ),          
       
     ],color = 'light')], style={'width': '33%', 'float': 'center', 'display': 'inline-block' ,'padding': '10px 0px 0px 7px'}),
#             dcc.Dropdown(
#                 id='month_column',
#                 options=[{'label': i, 'value': i} for i in multi_months_indicator],
#                 value='All',
#                 multi = True
#             ),           
#         ], style={'width': '33%', 'float': 'center', 'display': 'inline-block' ,'padding': '0px 0px 0px 10px'}),
        
            html.Div([
            dbc.Row([
                
                dbc.Col(html.Div([
        dbc.Card([
            html.H5("Complaints per State", className="card-title"),
         dcc.Graph(id = 'state_map'),
       
     ])],
    style={'backgroundColor': '#FFFFFF','height':'35%','width':'100%','padding': '10px 20px 30px 20px'}),width = "auto"),
                
                dbc.Col(html.Div([
        dbc.Card([
            html.H5("Trend Chart", className="card-title"),
        dcc.Graph(id = 'trend_map',config={
         'displayModeBar': False
     }),
       
     ])],
    style={'height':'12%','padding': '10px 20px 30px 20px'}),width = "auto"),
                
                
            ]),
            
            
            dbc.Row([
                
                dbc.Col(html.Div([
        dbc.Card([
            html.H5("Product, Sub Products", className="card-title"),
         dcc.Graph(id = 'tree_map'),
       
     ])],
    style={'height':'12%','padding': '10px 20px 30px 20px'}),width = "auto"),
                
                dbc.Col(html.Div([
        dbc.Card([
            html.H5("Issues, Sub Issues", className="card-title"),
        dcc.Graph(id = 'sunburst'),
       
     ])],
    style={'height':'12%','padding': '10px 20px 30px 20px'}),width = "auto"),
                
                
            ])
            
            
            ])]),
                
                
#     dcc.Tab(label='Finding the Product with Issue?', children=[
#             html.Div([
#             dbc.Row([
                
#                 dbc.Col(html.Div([
#         dbc.Card([
#          dcc.Graph(id = 'tree_map'),
       
#      ])],
#     style={'height':'12%','padding': '10px 20px 30px 20px'}),width = "auto"),
                
#                 dbc.Col(html.Div([
#         dbc.Card([
#         dcc.Graph(id = 'sunburst'),
       
#      ])],
#     style={'height':'12%','padding': '10px 20px 30px 20px'}),width = "auto"),
                
                
#             ])
    
   
    
# ])]),
    
    
    
    
    ],vertical = False)])
#,style={'margin-top':'20px','width': '250px', 'height': '50px', 'text-align':'center', 'margin-bottom': '30px','background-color': '#313131','color': '#d6d6d6','font-size':'1.25rem','font-family': 'sans-serif'}



@app.callback(
    dash.dependencies.Output('stackedbarchart', 'figure'),
    [
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     #dash.dependencies.Input('corona-column', 'value'),
     #dash.dependencies.Input('button', 'n_clicks'),
     ])
def update_graph(xaxis_column_name, yaxis_column_name):
    global button_clicks
    #months = 'Mar 2020'
    product_list = list(df['Product'].unique())
    fig = make_subplots(rows=1, cols=2,subplot_titles=("Product Count","Product Increase"))
    #if ('All' in xaxis_column_name) or (n_click-button_clicks==1):
    if ('All' in xaxis_column_name):
        dff = df
        #dff =dff[(dff['Year']>=year_value[0]) & (dff['Year']<=year_value[1])]
   
    else:
        #dff =df[(df['Year']>=year_value[0]) & (df['Year']<=year_value[1])]
        dff = df[df['State'].isin(xaxis_column_name)]
    
    
    if ('All' == yaxis_column_name):
        dff = dff
        
    else:
        #dff = df
        #months = yaxis_column_name
        dff = dff[dff['Month']==yaxis_column_name]
    
    
        
    product1 = []
    value1 = []
    for j in range(len(dff['Product'].value_counts())):
        product1.append(dff['Product'].value_counts().index[j])
        value1.append(dff['Product'].value_counts()[j])
    color = random.randint(100000,999999)
    diff = list(set(product_list) - set(product1))
    product1 = product1+diff
    value1 = value1+[0]*len(diff)
    product1.reverse()
    value1.reverse()
    value_percentage = np.array(value1)
    value_percentage = np.round((value_percentage/sum(value_percentage))*100,0)
    value_percentage = [int(v) for v in value_percentage]
    value = []

    for v in value_percentage:
        value.append(str(v)+'%')
    fig.add_trace(go.Bar(y=product1,
                x=value1,
                name=yaxis_column_name,orientation = 'h',
                text = value,textposition='auto',marker_color='#'+str(color)),row=1,col=1)

    fig.update_layout(barmode='group',width =1500,height = 800,plot_bgcolor="#FFFFFF",margin=dict(t=5, l=10,r=0,b=0),
                paper_bgcolor="#FFFFFF",font=dict(color="#000000",size = 10),showlegend=False)
    fig.update_xaxes(showgrid = False,visible = False)
    fig.update_yaxes(showgrid = False,visible = True)

    if yaxis_column_name=='All':
        month = 'Mar 2020'
    else:
        month = yaxis_column_name
    other_month = month[:3]+' 2019'
    my_months = [month,other_month]


    dff1 = df[df['Month']== month]
    dff2 = df[df['Month']== other_month]
    #print(dff1)

    product2 = []
    value2 = []
    for i in range(len(dff1['Product'].value_counts())):
        product2.append(dff1['Product'].value_counts().index[i])
        value2.append(dff1['Product'].value_counts()[i])

    first_dict =  dict(zip(product2, value2))


    product3 = []
    value3 = []
    for i in range(len(dff2['Product'].value_counts())):
        product3.append(dff2['Product'].value_counts().index[i])
        value3.append(dff2['Product'].value_counts()[i])

    second_dict =  dict(zip(product3, value3))

    relative_value = []
    prods = []
    for i in range(len(first_dict)):
        prod = list(first_dict.keys())[i]
        prods.append(prod)
        relative_value.append((first_dict[prod]-second_dict[prod])*100/second_dict[prod])

    relative_dict = dict(zip(prods,np.round(np.array(relative_value),1)))

    product = list(relative_dict.keys())
    value = list(relative_dict.values())
    product.reverse()
    value.reverse()
    colors = []
    for i in value:
        if i>0:
            colors.append('#32a852')
        else:
            colors.append('#cc1b38')
    values = []
    for v in value:
        values.append(str(int(v))+'%')
    fig.add_trace((go.Bar(
        name="Relative Increase/Decrease YYT",orientation = "h",
        y = product,
        x = value,
        textposition = "auto",
        text = values,
        marker_color=colors
        )),row=1,col=2)
    fig.update_yaxes(showgrid = False,visible = False,row=1,col=2)

    fig.update_layout(height=400, width=650,margin = dict(t =50,r=0,b=0,l=50),showlegend = False)


    #return fig
#     if (n_click-button_clicks==1):
#         dff = df
        #dff =dff[(dff['Year']>=year_value[0]) & (dff['Year']<=year_value[1])]

    #button_clicks = n_click
    fig.write_image('Product YTD.png')
    return fig


@app.callback(
    dash.dependencies.Output('bar_line', 'figure'),
    [
     dash.dependencies.Input('first_months_column', 'value'),
     dash.dependencies.Input('second_months_column', 'value'),
     #dash.dependencies.Input('button', 'n_clicks'),
     ])
def update_graph(first_months_column,second_months_column):
    global button_clicks
    
#     if two_months_column=='All':
#         two_months_column=['Mar 2020','Feb 2020']
    
#     two_months_column.sort(key = lambda date: datetime.strptime(date, '%b %Y')) 
    
    fig = go.Figure()
    #if ('All' in xaxis_column_name) or (n_click-button_clicks==1):
#     if ('All' in xaxis_column_name):
#         dff = df
#         #dff =dff[(dff['Year']>=year_value[0]) & (dff['Year']<=year_value[1])]
   
#     else:
#         #dff =df[(df['Year']>=year_value[0]) & (df['Year']<=year_value[1])]
#         dff = df[df['State'].isin(xaxis_column_name)]
    
    
#     if ('All' == two_months_column):
#         dff = dff
        
#     else:
    dff1 = df[df['Month']== first_months_column]
    dff2 = df[df['Month']== second_months_column]

    product2 = []
    value2 = []
    for i in range(len(dff1['Product'].value_counts())):
        product2.append(dff1['Product'].value_counts().index[i])
        value2.append(dff1['Product'].value_counts()[i])

    first_dict =  dict(zip(product2, value2))


    product3 = []
    value3 = []
    for i in range(len(dff2['Product'].value_counts())):
        product3.append(dff2['Product'].value_counts().index[i])
        value3.append(dff2['Product'].value_counts()[i])

    second_dict =  dict(zip(product3, value3))

    relative_value = []
    prods = []
    for i in range(len(first_dict)):
        prod = list(first_dict.keys())[i]
        prods.append(prod)
        relative_value.append((first_dict[prod]-second_dict[prod]))

    relative_dict = dict(zip(prods,np.round(np.array(relative_value),1)))

    product = list(relative_dict.keys())
    value = list(relative_dict.values())
    product.reverse()
    value.reverse()

    fig = go.Figure(go.Waterfall(name = "20", orientation = "v",
    measure = ["absolute","relative", "relative", "relative", "relative", "relative", "relative","relative","relative","relative",'absolute'],
    x = [second_months_column]+list(relative_dict.keys())+[first_months_column],
    textposition = "outside",
    text =[dff2.shape[0]]+list(relative_dict.values())+[dff1.shape[0]],
    y = [dff2.shape[0]-int(0.95*(dff2.shape[0]))]+list(relative_dict.values())+[dff1.shape[0]-int(0.95*(dff2.shape[0]))],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
    ))

    fig.update_yaxes(showgrid = False,visible = False)

    fig.update_layout(height=500, width=600,margin = dict(t =5,r=0,b=0,l=10),plot_bgcolor="#FFFFFF",
                paper_bgcolor="#FFFFFF",font=dict(color="#000000",size = 12))
    #return fig

    
#     if (n_click-button_clicks==1):
#         dff = df
#         #dff =dff[(dff['Year']>=year_value[0]) & (dff['Year']<=year_value[1])]

#     button_clicks = n_click
    fig.write_image('Two Month Comparision.png')
    return fig

@app.callback(
    dash.dependencies.Output('tab2_table', 'figure'),
    [
     dash.dependencies.Input('first_months_column', 'value'),
     dash.dependencies.Input('second_months_column', 'value'),
     ])
def update_graph(first_months_column,second_months_column):
    
    
    fig = go.Figure()

    dff1 = df[df['Month']== first_months_column]
    dff2 = df[df['Month']== second_months_column]
    products = df['Product'].unique()

    product_column = []
    issue_column = []
    count_column = []

    for product in range(len(products)):
        first_data_products = []
        first_data_counts = []
        for i in range(len(dff1[dff1['Product']==products[product]]['Issue'].value_counts())):
            first_data_products.append(dff1[dff1['Product']==products[product]]['Issue'].value_counts().index[i])
            first_data_counts.append(dff1[dff1['Product']==products[product]]['Issue'].value_counts()[i])

        second_data_products = []
        second_data_counts = []
        for i in range(len(dff2[dff2['Product']==products[product]]['Issue'].value_counts())):
            second_data_products.append(dff2[dff2['Product']==products[product]]['Issue'].value_counts().index[i])
            second_data_counts.append(dff2[dff2['Product']==products[product]]['Issue'].value_counts()[i])

        both_products_list = list(set(first_data_products+second_data_products))
        first_data_dict = dict(zip(first_data_products, first_data_counts))
        second_data_dict = dict(zip(second_data_products,second_data_counts))
        issue = []
        count = []
        for i in range(len(both_products_list)):
            issue.append(both_products_list[i])
            try:
                value1 = first_data_dict[both_products_list[i]]
            except:
                value1 = 0
            #print(value1)
            try:
                value2 = second_data_dict[both_products_list[i]]
            except:
                value2 = 0

            count.append((value1-value2))
        final_dict = dict(zip(issue,count))
        sorted_final_dict = dict(sorted(final_dict.items(), key=operator.itemgetter(1),reverse = True))
        product_column.append(products[product])
        issue_column.append('<br>'.join(list(sorted_final_dict.keys())[:3]))
        sorted_values = [str(v) for v in list(sorted_final_dict.values())[:3]]
        count_column.append('<br>'.join(sorted_values))

    values = [product_column, #1st col
    issue_column,count_column]


    fig = go.Figure(data=[go.Table(
      columnorder = [1,2,3],
      columnwidth = [80,400,100],
      header = dict(
        values = [['<b>PRODUCT</b>'],
                      ['<b>ISSUE</b>'],['<b>COUNT</b>']],
        line_color='darkslategray',
        fill_color='royalblue',
        align=['center','center'],
        font=dict(color='white', size=12),
        height=40
      ),
      cells=dict(
        values=values,
        line_color='darkslategray',
        fill=dict(color=['paleturquoise', 'white']),
        align=['left', 'left','center'],
        font_size=12,
        height=30)
        )
    ])
    fig.update_layout(width =950,height = 650,plot_bgcolor="#FFFFFF",margin=dict(t=0, l=0,r=0,b=0),
            paper_bgcolor="#FFFFFF",font=dict(color="#000000"),showlegend = False)
    fig.write_image('Tab 2 Table with Issues and Sub Issues.png')
    fig.update_layout(width =800,height = 500,plot_bgcolor="#FFFFFF",margin=dict(t=0, l=0,r=0,b=0),
            paper_bgcolor="#FFFFFF",font=dict(color="#000000"),showlegend = False)
    return fig
    
    
    
    
@app.callback(
    dash.dependencies.Output('wordcloud_corona', 'src'),
    [
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     
     ])
def update_graph(xaxis_column_name, yaxis_column_name):
    global button_clicks
    
    if ('All' in xaxis_column_name):
        dff = df
   
    else:
        dff = df[df['State'].isin(xaxis_column_name)]
    
    if ('All' == yaxis_column_name):
        dff = dff
        
    else:
        dff = dff[dff['Month']==yaxis_column_name]
    dff =dff[dff['Corona Flag']=='Corona Complaint']
    
    ALL_STOPWORDS = set(list(STOPWORDS)+['unknown'])

    wordcloud = WordCloud(width = 200,height= 200,stopwords=ALL_STOPWORDS).generate(str(' '.join(dff['Total Clean'].astype(str))))  
    file_name = random.randint(0,1000)
    wordcloud.to_file('assets/'+str(file_name)+'.png')
    files = glob.glob('C:\\Users\\samarth152721\\Desktop\\POC\\*.jpg')
    for i in files:
        os.remove(i)
    wordcloud.to_file(str(file_name)+'.jpg')
    #button_clicks_wordcloud = n_click
    return '/assets/'+str(file_name)+'.png'
    
    
@app.callback(
    dash.dependencies.Output('wordcloud_non_corona', 'src'),
    [
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     
     ])
def update_graph(xaxis_column_name, yaxis_column_name):
    global button_clicks
    files = glob.glob('assets/*.png')
    for i in files:
        os.remove(i)
    if ('All' in xaxis_column_name):
        dff = df
   
    else:
        dff = df[df['State'].isin(xaxis_column_name)]
    
    if ('All' == yaxis_column_name):
        dff = dff
        
    else:
        dff = dff[dff['Month']==yaxis_column_name]
    dff =dff[dff['Corona Flag']=='Non Corona Complaint']
    
    ALL_STOPWORDS = set(list(STOPWORDS)+['unknown'])

    wordcloud = WordCloud(width = 200,height= 200,stopwords=ALL_STOPWORDS).generate(str(' '.join(dff['Total Clean'].astype(str))))  
    file_name = random.randint(0,1000)
    wordcloud.to_file('assets/'+str(file_name)+'.png')
    files = glob.glob('C:\\Users\\samarth152721\\Desktop\\POC\\*.jpeg')
    for i in files:
        os.remove(i)
    wordcloud.to_file(str(file_name)+'.jpeg')

    #button_clicks_wordcloud = n_click
    return '/assets/'+str(file_name)+'.png'


@app.callback(
    dash.dependencies.Output('total_complaints', 'children'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     ])
def update_graph(xaxis_column_name,yaxis_column_name):
    
    if ('All' in xaxis_column_name):
        dff = df   
    else:
        dff = df[df['State'].isin(xaxis_column_name)]
    
    if ('All' == yaxis_column_name):
        dff = dff
        
    else:
        dff = dff[dff['Month']==yaxis_column_name]

    t = str(dff.shape[0])
    if len(t)>3:
        return (re.sub(t[-3:],'',t)+','+t[-3:])
    else:
        return t

@app.callback(
    dash.dependencies.Output('number_of_corona_complaints', 'children'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     ])
def update_graph(xaxis_column_name,yaxis_column_name):
    
    if ('All' in xaxis_column_name):
        dff = df   
    else:
        dff = df[df['State'].isin(xaxis_column_name)]
    
    if ('All' == yaxis_column_name):
        dff = dff
        
    else:
        dff = dff[dff['Month']==yaxis_column_name]
        
    dff=dff[dff['Corona Flag']=='Corona Complaint']
    t = str(dff.shape[0])
    if len(t)>3:
        return (re.sub(t[-3:],'',t)+','+t[-3:])
    else:
        return t


@app.callback(
    dash.dependencies.Output('number_of_non_corona_complaints', 'children'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     ])
def update_graph(xaxis_column_name,yaxis_column_name):
    
    if ('All' in xaxis_column_name):
        dff = df   
    else:
        dff = df[df['State'].isin(xaxis_column_name)]
    
    if ('All' == yaxis_column_name):
        dff = dff
        
    else:
        dff = dff[dff['Month']==yaxis_column_name]
        
    dff=dff[dff['Corona Flag']=='Non Corona Complaint']
    t = str(dff.shape[0])
    if len(t)>3:
        return (re.sub(t[-3:],'',t)+','+t[-3:])
    else:
        return t


@app.callback(
    dash.dependencies.Output('state_map', 'figure'),
    [dash.dependencies.Input('state_column', 'value'),
     dash.dependencies.Input('product_column', 'value'),
     dash.dependencies.Input('month_column','value')
     ])
def update_graph1(state_column,product_column,month_column):
    
    if ('All' in state_column):
        dff = df
       
    else:
        dff = df[df['State'].isin(state_column)]
    
    
    if ('All' in product_column):
        dff = dff
        
    else:
        dff = dff[dff['Product'].isin(product_column)]
        
    if ('All' in month_column):
        dff = dff
    else:
        dff = dff[dff['Month'].isin(month_column)]

    states = []
    values = []
    for i in range(len(dff['State'].value_counts())):
        states.append(dff['State'].value_counts().index[i])
        values.append(dff['State'].value_counts()[i])
   
    data = dict(type = 'choropleth', 
  
            # location: Arizoana, California, Newyork 
            locations = states, 
            
            # States of USA 
            locationmode = 'USA-states', 
              
            # colorscale can be added as per requirement 
            colorscale = 'tealgrn', 
            
            # text can be given anything you like 
            text = states, 
            z = values, 
            showlegend=False,
            showscale = True
            #colorbar = {'title': 'Number of Complaints'}
            ) 
    #colorbar = {'title': 'Number of Complaints'}
              
    layout = dict(geo ={'scope': 'usa'},clickmode = 'event+select') 

    fig = go.Figure(data = [data], layout = layout) 
    
    fig.update_layout(width = 600,height = 250,margin=dict(t=0, l=0,r=0,b=0),plot_bgcolor="#FFFFFF",
        paper_bgcolor="#FFFFFF",geo=dict(bgcolor= 'rgba(0,0,0,0)'),font=dict(color="#000000"))
    fig.update_layout(coloraxis_showscale=True)
    fig.write_image('State Map.png')
    return fig


@app.callback(
    dash.dependencies.Output('trend_map', 'figure'),
    [dash.dependencies.Input('state_column', 'value'),
     dash.dependencies.Input('product_column', 'value'),
     ])
def update_graph1(state_column,product_column):

    if 'All' in state_column:
        dff = df
       
    else:
        dff = df[df['State'].isin(state_column)]
    
    
    if ('All' in product_column):
        dff = dff
        
    else:
        dff = dff[dff['Product'].isin(product_column)]

        
    fig = go.Figure()
    
    products = dff['Product'].unique()

    for prod in products:
        new_df = dff[dff['Product']==prod]
        mon = []
        value = []
        for i in range(len(new_df['Month'].value_counts())):
            mon.append(new_df['Month'].value_counts().index[i])
            value.append(new_df['Month'].value_counts()[i])
        my_dict = dict(zip(mon,value))
        mon.sort(key = lambda date: datetime.strptime(date, '%b %Y')) 
        values = []
        for i in mon:
            values.append(my_dict[i])

        fig.add_trace(go.Scatter(x=mon, y=values,name = prod,stackgroup='one',
                        mode='none'
                        ))

        fig.update_layout(width = 800,height = 250,plot_bgcolor="#FFFFFF",margin=dict(t=0, l=0,r=0,b=0),
                paper_bgcolor="#FFFFFF",font=dict(color="#000000"))
        fig.update_yaxes(showgrid = False)
    fig.write_image('Trend Map.png')
    return fig
        
@app.callback(
    dash.dependencies.Output('tree_map', 'figure'),
    [dash.dependencies.Input('state_column', 'value'),
     dash.dependencies.Input('product_column', 'value'),
     dash.dependencies.Input('month_column','value')])
def update_graph1(state_column,product_column,month_column):

    if 'All' in state_column:
        dff = df
       
    else:
        dff = df[df['State'].isin(state_column)]
    
    
    if ('All' in product_column):
        dff = dff
        
    else:
        dff = dff[dff['Product'].isin(product_column)]

        
    if ('All' in month_column):
        dff = dff
    else:
        dff = dff[dff['Month'].isin(month_column)]
        
    
    dff['Complaints'] = 'Total Complaints' 
    fig = px.treemap(dff, path=['Complaints','Product', 'Sub-product'],color ='Product',color_continuous_scale='RdBu',color_discrete_map={'Credit reporting, credit repair services, or other personal consumer reports':'pink', 'Credit card or prepaid card':'teal'})
        
    fig.update_layout(width = 900,height = 350,plot_bgcolor="#FFFFFF",margin=dict(t=20, l=0,r=0,b=0),
            paper_bgcolor="#FFFFFF",font=dict(color="#000000"),showlegend = False,clickmode = 'event+select')
    fig.update_xaxes(showgrid = False)
    fig.update_yaxes(showgrid = False)
    fig.write_image('Product Tree Map.png')
    return fig
        
        
@app.callback(
    dash.dependencies.Output('sunburst', 'figure'),
    [dash.dependencies.Input('state_column', 'value'),
     dash.dependencies.Input('product_column', 'value'),
     dash.dependencies.Input('month_column','value'),
     dash.dependencies.Input('tree_map', 'clickData')])
def update_graph1(state_column,product_column,month_column,treedata):

    if 'All' in state_column:
        dff = df
       
    else:
        dff = df[df['State'].isin(state_column)]
    
    
    if ('All' in product_column):
        dff = dff
        
    else:
        dff = dff[dff['Product'].isin(product_column)]

        
    if ('All' in month_column):
        dff = dff
    else:
        dff = dff[dff['Month'].isin(month_column)]
        
   
    if (treedata == None) or (treedata['points'][0]['customdata'][0]=='(?)'):
        dff = dff
    else:
        dff = dff[dff['Product']==treedata['points'][0]['customdata'][0]]
        
    fig = px.sunburst(dff, path=['Issue', 'Sub-issue'])
    
    fig.update_layout(width =500,height = 350,plot_bgcolor="#FFFFFF",margin=dict(t=0, l=0,r=0,b=0),
            paper_bgcolor="#FFFFFF",font=dict(color="#000000"),showlegend = False)
    
    fig.update_yaxes(showgrid = False)
    fig.write_image('Sub Issue Sunburst.png')
    return fig


@app.callback(
    dash.dependencies.Output('table1', 'figure'),
    [dash.dependencies.Input('state_column', 'value'),
     dash.dependencies.Input('product_column', 'value'),
     dash.dependencies.Input('month_column','value')
     ])
def update_graph1(state_column,product_column,month_column):
    if ('All' in state_column):
        dff = df
       
    else:
        dff = df[df['State'].isin(state_column)]


    if ('All' in product_column):
        dff = dff

    else:
        dff = dff[dff['Product'].isin(product_column)]

    if ('All' in month_column):
        dff = dff
    else:
        dff = dff[dff['Month'].isin(month_column)]

    table_product = []
    issue_column = []
    count_column = []
    products = dff['Product'].unique()
    for prod in range(len(products)):
        dff1 = dff[dff['Product']==products[prod]]
        product = []
        value = []
        for i in range(len(dff1['Issue'].value_counts())):
            product.append(dff1['Issue'].value_counts().index[i])
            value.append(str(dff1['Issue'].value_counts()[i]))
        table_product.append(products[prod])
        issue_column.append('<br>'.join(product[:3]))
        count_column.append('<br>'.join(value[:3]))
        
    values = [table_product, #1st col
    issue_column,count_column]


    fig = go.Figure(data=[go.Table(
      columnorder = [1,2,3],
      columnwidth = [80,400,100],
      header = dict(
        values = [['<b>PRODUCT</b>'],
                      ['<b>ISSUE</b>'],['<b>COUNT</b>']],
        line_color='darkslategray',
        fill_color='royalblue',
        align=['center','center'],
        font=dict(color='white', size=12),
        height=40
      ),
      cells=dict(
        values=values,
        line_color='darkslategray',
        fill=dict(color=['paleturquoise', 'white']),
        align=['left', 'left','center'],
        font_size=12,
        height=30)
        )
    ])
    fig.update_layout(width =850,height=700,plot_bgcolor="#FFFFFF",margin=dict(t=0, l=0,r=0,b=0),
                paper_bgcolor="#FFFFFF",font=dict(color="#000000"),showlegend = False)
    fig.write_image('Tab 3 Products and Issue Count Table.png')
    return None



@app.callback(
    dash.dependencies.Output("modal", "is_open"),
    [dash.dependencies.Input("open", "n_clicks"), dash.dependencies.Input("send", "n_clicks")],
    [dash.dependencies.State("modal", "is_open")],
)
def toggle_modal(n1, n2,is_open):
    if n1 or n2:
        return not is_open
    return is_open

@app.callback(
    dash.dependencies.Output("dummy", "value"),
    [dash.dependencies.Input("send","n_clicks"),dash.dependencies.Input("email", "value"),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     #dash.dependencies.Input('corona-column', 'value'),
     dash.dependencies.Input('first_months_column', 'value'),
     dash.dependencies.Input('second_months_column', 'value'),
     dash.dependencies.Input('state_column', 'value'),
     dash.dependencies.Input('product_column', 'value'),
     dash.dependencies.Input('month_column','value'),
     dash.dependencies.Input('tree_map', 'clickData')
    ],
)
def toggle_modal(close,email,xaxis_column_name,yaxis_column_name,first_months_column,second_months_column,state_column,product_column,month_column,treemap_clickdata):
    if close:
        
        if ('All' in xaxis_column_name):
            dff1 = df
        
        else:
            dff1 = df[df['State'].isin(xaxis_column_name)]

        if ('All' == yaxis_column_name):
            dff1 = dff1

        else:
            dff1 = dff1[dff1['Month']==yaxis_column_name]

        
        
        dff_1 = df[df['Month']== first_months_column]
        dff_2 = df[df['Month']== second_months_column]
        
        dff2 = pd.concat([dff_1,dff_2])
        
        
        if 'All' in state_column:
            dff3 = df

        else:
            dff3 = df[df['State'].isin(state_column)]


        if ('All' in product_column):
            dff3 = dff3

        else:
            dff3 = dff3[dff3['Product'].isin(product_column)]


        if ('All' in month_column):
            dff3 = dff3
        else:
            dff3 = dff3[dff3['Month'].isin(month_column)]
        dff1.reset_index(inplace = True,drop = True)
        writer = pd.ExcelWriter('Filtered Data.xlsx', engine='xlsxwriter')
        dff1.to_excel(writer, sheet_name='1. YTD Comparision')
        dff2.to_excel(writer, sheet_name='2. Two Months Together')
        dff3.to_excel(writer, sheet_name = '3. Month on Month Deep Dive')
        writer.save()
        
        
        generate_report(xaxis_column_name,yaxis_column_name,first_months_column,second_months_column,state_column,
                        product_column,month_column,treemap_clickdata)
               
        
        emails = email.split(',')
        
        
        files = ['C:\\Users\\samarth152721\\Desktop\\POC\\Report.docx','C:\\Users\\samarth152721\\Desktop\\POC\\Filtered Data.xlsx']
        pythoncom.CoInitialize()
        outlook = win32com.client.Dispatch('outlook.application')
        #pythoncom.CoInitialize()
        for ema in emails:
            mail = outlook.CreateItem(0)
            mail.To = ema
            mail.Subject = 'Complaints Dashboard Results'
            #mail.Body = 'Complaints Dashboard Results'
            mail.HTMLBody = mail.HTMLBody = 'Hi {}, <br> <br> Thank you for using Complaints Dashboard. Please find attached Filtered data and Report Generated.<br><br> Thank you, <br> Dashboard Team.'.format(ema.split('.')[0].title()) #this field is optional

        # To attach a file to the email (optional):

            for attachment in files:
                mail.Attachments.Add(attachment)

            mail.Send()
            
        print('Mail Sent')

In [2]:
def generate_report(xaxis_column_name,yaxis_column_name,first_months_column,second_months_column,state_column,
                    product_column,month_column,treemap_clickdata):
    if ('All' in xaxis_column_name):
            dff1 = df
        
    else:
        dff1 = df[df['State'].isin(xaxis_column_name)]

    if ('All' == yaxis_column_name):
        dff1 = dff1

    else:
        dff1 = dff1[dff1['Month']==yaxis_column_name]

    
        
    
    product1 = []
    value1 = []
    for j in range(len(dff1['Product'].value_counts())):
        product1.append(dff1['Product'].value_counts().index[j])
        value1.append(dff1['Product'].value_counts()[j])
    color = random.randint(100000,999999)
    #product1.reverse()
    #value1.reverse()
    
    value_percentage = np.array(value1)
   
    value_percentage = np.round((value_percentage/sum(value_percentage))*100,1)
    
    value = []
    for v in value_percentage:
        value.append(str(v)+'%')
    value2 = []
    for v in value1:
        value2.append(str(v))
    
    states = xaxis_column_name
    month = yaxis_column_name
    text_tab1_multigraph = 'This chart shows percentage share of top three products for {} in {} month/s. The products {} are having {} number of complaints respectively. The total number of complaints are {}.'.format(', '.join(states),month,', '.join(product1[:3]),', '.join(value2[:3]),str(sum(value1)))
    text_corona_wordcloud = 'This is the word cloud for Covid Complaints coming from {} states in {} month/s'.format(', '.join(states),month)
    text_non_corona_wordcloud = 'This is the word cloud for Non - Covid Complaints coming from {} states in {} month/s'.format(', '.join(states),month)
    
    dff1 = df[df['Month']== first_months_column]
    dff2 = df[df['Month']== second_months_column]

    product2 = []
    value2 = []
    for i in range(len(dff1['Product'].value_counts())):
        product2.append(dff1['Product'].value_counts().index[i])
        value2.append(dff1['Product'].value_counts()[i])

    first_dict =  dict(zip(product2, value2))


    product3 = []
    value3 = []
    for i in range(len(dff2['Product'].value_counts())):
        product3.append(dff2['Product'].value_counts().index[i])
        value3.append(dff2['Product'].value_counts()[i])

    second_dict =  dict(zip(product3, value3))

    relative_value = []
    prods = []
    for i in range(len(first_dict)):
        prod = list(first_dict.keys())[i]
        prods.append(prod)
        relative_value.append((first_dict[prod]-second_dict[prod]))

    relative_dict = dict(zip(prods,np.round(np.array(relative_value),1)))
    sorted_relative_dict = sorted(relative_dict.items(), key=operator.itemgetter(1),reverse = True)
    text_two_month_comparision = 'Representing number of complaints increasing or decreasing in {} as compared to {}'.format(first_months_column,second_months_column)
    
    document = Document()

    heading = document.add_heading('Dashboard Report', 0)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER


    #Tab 1
    heading = document.add_heading(text_tab1_multigraph, level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture('Product YTD.png', width=Inches(5.25))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

    heading = document.add_heading(text_corona_wordcloud, level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture(glob.glob('*.jpg')[0], width=Inches(5.25),height = Inches(3))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

    heading = document.add_heading(text_non_corona_wordcloud, level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture(glob.glob('*.jpeg')[0], width=Inches(5.25),height = Inches(3))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

    #Tab 2
    heading = document.add_heading(text_two_month_comparision, level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture(glob.glob('Two Month Comparision.png')[0], width=Inches(5.25),height = Inches(5))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    text_two_month_comparision = 'Product with Issue count for {} as compared to {}'.format(first_months_column,second_months_column)
    heading = document.add_heading(text_two_month_comparision, level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture(glob.glob('Tab 2 Table with Issues and Sub Issues.png')[0], width=Inches(5.25),height = Inches(5))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    #Tab 3
        
    tab3_states_bullet = 'States selected are {}'.format(', '.join(state_column)) 
    tab3_product_bullet = 'Products selected are {}'.format(', '.join(product_column))
    tab3_month_bullet = 'Months selected are {}'.format(', '.join(month_column))
    
    heading = document.add_heading('The filters applied to the following charts are:-',level=3)
    document.add_paragraph(tab3_states_bullet, style='List Number')
    document.add_paragraph(tab3_product_bullet,style = 'List Number')
    document.add_paragraph(tab3_month_bullet,style = 'List Number')
    
    
    if 'All' in state_column:
        dff3 = df

    else:
        dff3 = df[df['State'].isin(state_column)]


    if ('All' in product_column):
        dff3 = dff3

    else:
        dff3 = dff3[dff3['Product'].isin(product_column)]


    if ('All' in month_column):
        dff3 = dff3
    else:
        dff3 = dff3[dff3['Month'].isin(month_column)]
        
    #dff3 = df[df['State'].isin(states)]
    #dff3['State'].value_counts()
    stat = []
    val= []
    for i in range(len(dff3['State'].value_counts())):
        stat.append(dff3['State'].value_counts().index[i])
        val.append(str(dff3['State'].value_counts()[i]))
    
    text_tab3_state_map = 'This is a geographical map depicting {} complaints coming from {} states respectively for {} in {}'.format(', '.join(val),', '.join(stat),', '.join(product_column),', '.join(month_column))
    heading = document.add_heading(text_tab3_state_map, level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture(glob.glob('State Map.png')[0], width=Inches(5.25),height = Inches(2))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    heading = document.add_heading('Trend Map starting from Jan 2019 till June 2020', level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture(glob.glob('Trend Map.png')[0], width=Inches(5.25),height = Inches(2))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    
    
    prod = []
    val = []
    for i in range(len(dff3['Product'].value_counts())):
        prod.append(dff3['Product'].value_counts().index[i])
        val.append(str(dff3['Product'].value_counts()[i]))

    if len(prod)>3:
        product_filter = prod[:3]
        value_filter = val[:3]
    else:
        product_filter = prod
        value_filter = val
        
    print(treemap_clickdata)
    if treemap_clickdata == None:
        text_tab3_tree_map = 'The below chart represents the number of complaints product wise. The maximum number of complaints are coming for {} with the number of complaints as {} respectively.'.format(', '.join(product_filter),', '.join(value_filter))
    else:    
        text_tab3_tree_map = 'The below chart represents the number of complaints product wise. The maximum number of complaints are coming for {} with the number of complaints as {} respectively. Within the tree map {} is also selected'.format(', '.join(product_filter),', '.join(value_filter),treemap_clickdata['points'][0]['customdata'][0])
    print(text_tab3_tree_map)
    heading = document.add_heading(text_tab3_tree_map, level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture(glob.glob('Product Tree Map.png')[0], width=Inches(5.25),height = Inches(2))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    sub_issue_prod = []
    sub_issue_value = []
    for i in range(len(product_filter)):
        dff_prod1 = dff3[dff3['Product']==product_filter[i]]
        sub_issue_prod.append(dff_prod1['Sub-issue'].value_counts().index[0])
        sub_issue_value.append(dff_prod1['Sub-issue'].value_counts()[0])
        
   
    heading = document.add_heading('The most frequently raised Issue within each product is:-', level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    for i in range(len(sub_issue_prod)):
        text = '{} are having an Issue of {} with a count of {}.'.format(product_filter[i],sub_issue_prod[i],sub_issue_value[i])
        heading = document.add_paragraph(text,style='List Bullet')
        
    picture = document.add_picture(glob.glob('Sub Issue Sunburst.png')[0], width=Inches(5.25),height = Inches(3))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    heading = document.add_heading('Product and Issue comparision for the filters selected in Tab 3', level=3)
    heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
    picture = document.add_picture(glob.glob('Tab 3 Products and Issue Count Table.png')[0], width=Inches(5.25),height = Inches(5))
    last_paragraph = document.paragraphs[-1] 
    last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    document.save('Report.docx')

In [3]:
#generate_report(['CA','FL'],'Mar 2020','All','Mar 2020','Feb 2020',['CA','NJ'],['Credit Cards','Mortgage'],['Mar 2020','April 2020'])

In [ ]:
if __name__ == '__main__':
    app.run_server(port = 5000,threaded = True)

Dash is running on http://127.0.0.1:5000/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2020 14:59:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 14:59:21] "GET /assets/style.css?m=1594303644.9988701 HTTP/1.1" 304 -
127.0.0.1 - - [28/Jul/2020 14:59:22] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 14:59:22] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 14:59:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 14:59:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 14:59:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 14:59:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 14:59:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:00:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:00:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:00:54] "POST /_dash-up

127.0.0.1 - - [28/Jul/2020 15:12:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2020 15:12:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [ ]:
from docx2pdf import convert

In [ ]:
convert("Report.docx", "C:/Users/samarth152721/Desktop/POC/output.pdf")